In [4]:
!pip install vaderSentiment

In [5]:
import pandas as pd
import os
import gc
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive')

PROJECT_PATH = "/content/drive/My Drive/Smart Complaint Prioritizer"
RAW_FILE_PATH = f"{PROJECT_PATH}/data/raw/complaints.csv"
PROCESSED_FILE_PATH = f"{PROJECT_PATH}/data/processed/complaints_subset.csv"
FINAL_OUTPUT_PATH = f"{PROJECT_PATH}/data/processed/complaints_with_features.csv"

if not os.path.exists(PROCESSED_FILE_PATH):
    print("Processed file not found. Generating it now from Raw data...")

    if not os.path.exists(RAW_FILE_PATH):
        print(f"CRITICAL ERROR: Raw file not found at {RAW_FILE_PATH}")
    else:
        COLUMN_MAPPING = {
            'Consumer complaint narrative': 'consumer_complaint_narrative',
            'Issue': 'issue',
            'Product': 'product',
            'Company': 'company',
            'Submitted via': 'submitted_via',
            'Date received': 'date_received',
            'Company public response': 'company_public_response',
            'Company response to consumer': 'company_response_to_consumer'
        }

        chunk_iterator = pd.read_csv(
            RAW_FILE_PATH,
            usecols=COLUMN_MAPPING.keys(),
            chunksize=100000,
            low_memory=False
        )

        first_chunk = True
        total_rows = 0

        for chunk in chunk_iterator:
            cleaned = chunk.dropna(subset=['Consumer complaint narrative'])
            if cleaned.empty: continue

            cleaned = cleaned.rename(columns=COLUMN_MAPPING)

            mode = 'w' if first_chunk else 'a'
            header = True if first_chunk else False
            cleaned.to_csv(PROCESSED_FILE_PATH, mode=mode, header=header, index=False)

            total_rows += len(cleaned)
            first_chunk = False
            del chunk, cleaned
            gc.collect()

        print(f"Data Ingestion Complete. Saved {total_rows} rows.")

else:
    print("Processed file already exists. Proceeding...")

print("Loading data for Feature Engineering...")
df = pd.read_csv(PROCESSED_FILE_PATH)
df = df.dropna(subset=['consumer_complaint_narrative'])

tqdm.pandas(desc="Cleaning Text")
def clean_text(text):
    if pd.isna(text):
        return ""

    text = str(text).lower()

    text = re.sub(r'\$\d+', 'money_token', text)
    text = re.sub(r'\d+\s+dollars', 'money_token', text)
    text = re.sub(r'\d+', 'number_token', text)
    text = re.sub(r'x{2,}', '', text)
    text = re.sub(r'[^a-z_ ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

df['cleaned_narrative'] = df['consumer_complaint_narrative'].progress_apply(clean_text)

analyzer = SentimentIntensityAnalyzer()

HIGH_KEYWORDS = [
    'fraud', 'theft', 'illegal', 'scam', 'threaten', 'harass', 'lawsuit', 'identit', 'arrest',
    'loan', 'mortgage', 'foreclosure', 'car', 'vehicle', 'repossession', 'collection', 'debt',
    'bankrupt', 'veteran', 'military', 'money_token'
]

MEDIUM_KEYWORDS = ['fee', 'charge', 'late', 'interest', 'mistake', 'error', 'billing', 'credit report', 'balance']

def get_priority_score(row):
    text = row['cleaned_narrative']
    vs = analyzer.polarity_scores(text)
    sentiment_score = vs['compound']

    text_words = set(text.split())
    has_high = any(word in text_words for word in HIGH_KEYWORDS)
    has_medium = any(word in text_words for word in MEDIUM_KEYWORDS)

    if has_high or sentiment_score < -0.60:
        return "High"
    elif has_medium or sentiment_score < -0.20:
        return "Medium"
    else:
        return "Low"

print("Calculating Priority Scores...")
tqdm.pandas(desc="Scoring Priority")
df['priority'] = df.progress_apply(get_priority_score, axis=1)

df.to_csv(FINAL_OUTPUT_PATH, index=False)
print(f"SUCCESS! Feature-rich data saved to: {FINAL_OUTPUT_PATH}")
print("FINAL PRIORITY DISTRIBUTION")
print(df['priority'].value_counts())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processed file already exists. Proceeding...
Loading data for Feature Engineering...


Cleaning Text: 100%|██████████| 383564/383564 [01:14<00:00, 5134.19it/s]


Calculating Priority Scores...


Scoring Priority: 100%|██████████| 383564/383564 [16:19<00:00, 391.50it/s]


SUCCESS! Feature-rich data saved to: /content/drive/My Drive/Smart Complaint Prioritizer/data/processed/complaints_with_features.csv
FINAL PRIORITY DISTRIBUTION
priority
High      257892
Low        67622
Medium     58050
Name: count, dtype: int64
